In [122]:
import polars.selectors as cs
import polars as pl
import plotly as plt
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from scipy.stats import f_oneway
from scipy.stats import tukey_hsd
from tqdm import tqdm
import xlsxwriter




In [93]:
parquet_path = "./parquet4visual.parquet"

In [94]:
df = pl.read_parquet(parquet_path)

In [95]:
pl.Config.set_tbl_cols(250)
df.head

<bound method DataFrame.head of shape: (449_537, 232)
┌─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬───

In [96]:
# select a subset of the dataframe to do statistical testing
stat_df = df.select([
    'competition_region_division', 
    'position_grouped',
    'P90 Distance',
    'P90 Running Distance',
    'P90 HSR Distance', 
    'P90 Sprinting Distance', 
    'PSV-99'
])

In [97]:
stat_df = stat_df.filter(pl.col("position_grouped") == "CB")

In [98]:
grouped = stat_df.group_by('competition_region_division').agg(
    pl.col('PSV-99').alias('PSV_99_values')
)
grouped

competition_region_division,PSV_99_values
str,list[f64]
"""Chile 1""","[28.9, 30.3, … 26.4]"
"""Slovakia 1""","[32.0, 26.4, … 29.0]"
"""Sweden 1""","[28.4, 28.4, … 27.6]"
"""Czech Republic 1""","[26.4, 26.8, … 28.9]"
"""Mexico 1""","[28.0, 29.7, … 28.5]"
…,…
"""Serbia 1""","[27.1, 28.6, … 28.2]"
"""Netherlands 2""","[26.6, 28.2, … 30.8]"
"""South Korea 1""","[28.0, 29.3, … 27.9]"


In [99]:
# Group by 'competition_region_division' and extract the corresponding values for each group
valid_groups = {
    value: stat_df.filter(pl.col("competition_region_division") == value)["PSV-99"].to_list()
    for value in stat_df["competition_region_division"].unique().to_list()
    if len(stat_df.filter(pl.col("competition_region_division") == value)) > 1
}

/tmp/ipykernel_17384/3376544247.py:5: UserWarning: Comparisons with None always result in null. Consider using `.is_null()` or `.is_not_null()`.
  if len(stat_df.filter(pl.col("competition_region_division") == value)) > 1


In [100]:
valid_groups

{'Ecuador 1': [30.6,
  27.2,
  29.7,
  27.8,
  29.6,
  28.0,
  25.9,
  28.5,
  27.6,
  30.1,
  28.1,
  28.9,
  29.8,
  28.3,
  26.6,
  30.1,
  27.4,
  25.8,
  29.5,
  24.2,
  26.6,
  30.5,
  28.0,
  31.6,
  27.9,
  25.7,
  27.2,
  29.3,
  30.2,
  28.0,
  31.3,
  27.2,
  27.4,
  27.1,
  29.7,
  26.9,
  24.9,
  28.3,
  27.6,
  29.6,
  26.8,
  27.7,
  29.9,
  28.9,
  28.5,
  26.7,
  32.3,
  28.8,
  24.1,
  31.1,
  27.4,
  24.8,
  27.0,
  27.7,
  29.5,
  29.4,
  29.6,
  29.4,
  27.8,
  27.2,
  28.8,
  25.2,
  25.8,
  26.9,
  27.9,
  29.3,
  28.1,
  27.0,
  29.5,
  29.5,
  29.9,
  26.9,
  31.1,
  27.7,
  27.2,
  27.9,
  32.2,
  28.6,
  25.2,
  28.9,
  31.5,
  25.9,
  30.7,
  25.9,
  26.6,
  26.9,
  31.3,
  26.2,
  27.4,
  28.9,
  26.5,
  27.0,
  27.4,
  26.0,
  29.1,
  26.7,
  27.4,
  28.3,
  28.5,
  30.9,
  27.8,
  31.2,
  26.5,
  28.3,
  30.3,
  26.7,
  27.2,
  31.5,
  26.8,
  27.4,
  28.7,
  27.0,
  29.5,
  28.4,
  27.9,
  32.2,
  29.1,
  31.0,
  28.3,
  26.8,
  30.7,
  26.8,
  30.1,
  2

In [ ]:
# Step 2: Run Tukey HSD on all data at once
group_names = list(valid_groups.keys())
result = tukey_hsd(*valid_groups.values())

# Get number of groups
n_groups = len(group_names)

# Loop through all pairs
for i in range(n_groups):
    for j in range(i + 1, n_groups):
        print(f"{group_names[i]} vs {group_names[j]}:")
        print(f"  Statistic: {result.statistic[i, j]:.4f}")
        print(f"  p-value: {result.pvalue[i, j]:.4f}")
        print(f"  Standard Error: {result._stand_err[i, j]:.4f}")
        print("-" * 40)

Ecuador 1 vs Chile 1:
  Statistic: -0.1367
  p-value: 1.0000
  Standard Error: 0.1341
----------------------------------------
Ecuador 1 vs Hungary 1:
  Statistic: -0.0443
  p-value: 1.0000
  Standard Error: 0.1187
----------------------------------------
Ecuador 1 vs South Africa 1:
  Statistic: -0.1408
  p-value: 1.0000
  Standard Error: 0.2048
----------------------------------------
Ecuador 1 vs Italy 2:
  Statistic: 0.2384
  p-value: 1.0000
  Standard Error: 0.1042
----------------------------------------
Ecuador 1 vs USA 3:
  Statistic: -0.4850
  p-value: 0.7933
  Standard Error: 0.1231
----------------------------------------
Ecuador 1 vs Spain 2:
  Statistic: -0.0536
  p-value: 1.0000
  Standard Error: 0.1108
----------------------------------------
Ecuador 1 vs France 2:
  Statistic: -0.1936
  p-value: 1.0000
  Standard Error: 0.1082
----------------------------------------
Ecuador 1 vs Mexico 1:
  Statistic: -0.5319
  p-value: 0.6012
  Standard Error: 0.1245
-----------------

In [133]:
# Step 2: Run Tukey HSD on all data at once
group_names = list(valid_groups.keys())
result = tukey_hsd(*valid_groups.values())

# Get number of groups
n_groups = len(group_names)

/home/ianh/miniconda3/envs/club_brugge/lib/python3.13/site-packages/scipy/integrate/_quadpack_py.py:1260: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,


In [ ]:
# get the unique positions


In [136]:
rows = []
# Loop through all pairs
for i in range(n_groups):
    for j in range(i + 1, n_groups):
        rows.append({
            "gn1": group_names[i],
            "gn2": group_names[j],
            "Statistic": round(result.statistic[i, j], 4),
            "P-value": round(result.pvalue[i, j], 4),
            "Standard Error": round(result._stand_err[i, j], 4),
        })


In [137]:
tukey_results_df = pl.DataFrame(rows)
print(tukey_results_df)

shape: (1_176, 5)
┌───────────┬────────────────┬───────────┬─────────┬────────────────┐
│ gn1       ┆ gn2            ┆ Statistic ┆ P-value ┆ Standard Error │
│ ---       ┆ ---            ┆ ---       ┆ ---     ┆ ---            │
│ str       ┆ str            ┆ f64       ┆ f64     ┆ f64            │
╞═══════════╪════════════════╪═══════════╪═════════╪════════════════╡
│ Ecuador 1 ┆ Chile 1        ┆ -0.1367   ┆ 1.0     ┆ 0.1341         │
│ Ecuador 1 ┆ Hungary 1      ┆ -0.0443   ┆ 1.0     ┆ 0.1187         │
│ Ecuador 1 ┆ South Africa 1 ┆ -0.1408   ┆ 1.0     ┆ 0.2048         │
│ Ecuador 1 ┆ Italy 2        ┆ 0.2384    ┆ 1.0     ┆ 0.1042         │
│ Ecuador 1 ┆ USA 3          ┆ -0.485    ┆ 0.7933  ┆ 0.1231         │
│ …         ┆ …              ┆ …         ┆ …       ┆ …              │
│ Austria 1 ┆ Greece 1       ┆ 0.2221    ┆ 1.0     ┆ 0.1298         │
│ Austria 1 ┆ Sweden 1       ┆ 0.3051    ┆ 0.9666  ┆ 0.0897         │
│ Croatia 1 ┆ Greece 1       ┆ 0.5604    ┆ 0.9795  ┆ 0.17           │
│ 

In [141]:
with xlsxwriter.Workbook("tukey_hsd_results.xlsx") as wb:
    tukey_results_df.write_excel(
        workbook = wb,
        worksheet = 'position',
        autofit = True,
        float_precision = 3,
        freeze_panes = (1,0),
        header_format = {"bold": True}
    )


vanaf hier ist niet meer relevant

In [130]:
with xlsxwriter.Workbook("match_benchmarks_stat_sep.xlsx") as wb:
    
    # Loop through all pairs
    for i in range(n_groups):
        for j in range(i + 1, n_groups):
            df = pl.DataFrame({
                "gn1": group_names[i],
                "gn2": group_names[j], 
                "Statistic": [round(result.statistic[i, j], 4)], 
                "P-value": [round(result.pvalue[i, j],4)], 
                "Standard Error": [round(result._stand_err[i, j],4)],
            })
            print(df)
            #print(f"{group_names[i]} vs {group_names[j]}:")
            #print(f"  Statistic: {result.statistic[i, j]:.4f}")
            #print(f"  p-value: {result.pvalue[i, j]:.4f}")
            #print(f"  Standard Error: {result._stand_err[i, j]:.4f}")
            print("-" * 40)

shape: (1, 5)
┌───────────┬─────────┬───────────┬─────────┬────────────────┐
│ gn1       ┆ gn2     ┆ Statistic ┆ P-value ┆ Standard Error │
│ ---       ┆ ---     ┆ ---       ┆ ---     ┆ ---            │
│ str       ┆ str     ┆ f64       ┆ f64     ┆ f64            │
╞═══════════╪═════════╪═══════════╪═════════╪════════════════╡
│ Ecuador 1 ┆ Chile 1 ┆ -0.1367   ┆ 1.0     ┆ 0.1341         │
└───────────┴─────────┴───────────┴─────────┴────────────────┘
----------------------------------------
shape: (1, 5)
┌───────────┬───────────┬───────────┬─────────┬────────────────┐
│ gn1       ┆ gn2       ┆ Statistic ┆ P-value ┆ Standard Error │
│ ---       ┆ ---       ┆ ---       ┆ ---     ┆ ---            │
│ str       ┆ str       ┆ f64       ┆ f64     ┆ f64            │
╞═══════════╪═══════════╪═══════════╪═════════╪════════════════╡
│ Ecuador 1 ┆ Hungary 1 ┆ -0.0443   ┆ 1.0     ┆ 0.1187         │
└───────────┴───────────┴───────────┴─────────┴────────────────┘
-----------------------------------

In [106]:
competitions = list(valid_groups.items())
competitions

[('Ecuador 1',
  [30.6,
   27.2,
   29.7,
   27.8,
   29.6,
   28.0,
   25.9,
   28.5,
   27.6,
   30.1,
   28.1,
   28.9,
   29.8,
   28.3,
   26.6,
   30.1,
   27.4,
   25.8,
   29.5,
   24.2,
   26.6,
   30.5,
   28.0,
   31.6,
   27.9,
   25.7,
   27.2,
   29.3,
   30.2,
   28.0,
   31.3,
   27.2,
   27.4,
   27.1,
   29.7,
   26.9,
   24.9,
   28.3,
   27.6,
   29.6,
   26.8,
   27.7,
   29.9,
   28.9,
   28.5,
   26.7,
   32.3,
   28.8,
   24.1,
   31.1,
   27.4,
   24.8,
   27.0,
   27.7,
   29.5,
   29.4,
   29.6,
   29.4,
   27.8,
   27.2,
   28.8,
   25.2,
   25.8,
   26.9,
   27.9,
   29.3,
   28.1,
   27.0,
   29.5,
   29.5,
   29.9,
   26.9,
   31.1,
   27.7,
   27.2,
   27.9,
   32.2,
   28.6,
   25.2,
   28.9,
   31.5,
   25.9,
   30.7,
   25.9,
   26.6,
   26.9,
   31.3,
   26.2,
   27.4,
   28.9,
   26.5,
   27.0,
   27.4,
   26.0,
   29.1,
   26.7,
   27.4,
   28.3,
   28.5,
   30.9,
   27.8,
   31.2,
   26.5,
   28.3,
   30.3,
   26.7,
   27.2,
   31.5,
   26.8,
   2

In [117]:
for competition1, metric_list1 in tqdm(valid_groups.items()):
    for competition2, metric_list2 in valid_groups.items():
        if competition1 != competition2:
            tres = tukey_hsd(metric_list1, metric_list2)
            print(
                f"{competition1} vs {competition2}:" 
                f"\nstatistic = {tres.statistic}," 
                f"\np-value = {tres.pvalue}",
                f"\n_nobs = {tres._nobs}",
                f"\n_ntreatments = {tres._ntreatments}",
                f"\n_stand_err = {tres._stand_err}",
            )

  0%|          | 0/49 [00:00<?, ?it/s]

Ecuador 1 vs Chile 1:
statistic = [[ 0.         -0.13670312]
 [ 0.13670312  0.        ]],
p-value = [[1.         0.48689992]
 [0.48689992 1.        ]] 
_nobs = 2 
_ntreatments = 335 
_stand_err = [[0.13531034 0.13888004]
 [0.13888004 0.14236025]]
Ecuador 1 vs Hungary 1:
statistic = [[ 0.         -0.04434544]
 [ 0.04434544  0.        ]],
p-value = [[1.         0.78715551]
 [0.78715551 1.        ]] 
_nobs = 2 
_ntreatments = 447 
_stand_err = [[0.12780397 0.11606425]
 [0.11606425 0.10299498]]
Ecuador 1 vs South Africa 1:
statistic = [[ 0.        -0.1407702]
 [ 0.1407702  0.       ]],
p-value = [[1.         0.62548233]
 [0.62548233 1.        ]] 
_nobs = 2 
_ntreatments = 221 
_stand_err = [[0.12995773 0.20364652]
 [0.20364652 0.25701128]]
Ecuador 1 vs Italy 2:
statistic = [[ 0.          0.23840292]
 [-0.23840292  0.        ]],
p-value = [[1.         0.11129655]
 [0.11129655 1.        ]] 
_nobs = 2 
_ntreatments = 821 
_stand_err = [[0.13255642 0.1057493 ]
 [0.1057493  0.06924322]]
Ecuador

  2%|▏         | 1/49 [00:00<00:27,  1.76it/s]

Ecuador 1 vs Netherlands 1:
statistic = [[ 0.          0.26474567]
 [-0.26474567  0.        ]],
p-value = [[1.         0.14729954]
 [0.14729954 1.        ]] 
_nobs = 2 
_ntreatments = 344 
_stand_err = [[0.1273825  0.12889004]
 [0.12889004 0.13038014]]
Ecuador 1 vs Denmark 1:
statistic = [[ 0.          0.21733053]
 [-0.21733053  0.        ]],
p-value = [[1.         0.17185451]
 [0.17185451 1.        ]] 
_nobs = 2 
_ntreatments = 465 
_stand_err = [[0.1252086  0.11230438]
 [0.11230438 0.09771057]]
Ecuador 1 vs Australia 1:
statistic = [[ 0.          1.17670455]
 [-1.17670455  0.        ]],
p-value = [[1.         0.00461106]
 [0.00461106 1.        ]] 
_nobs = 2 
_ntreatments = 198 
_stand_err = [[0.13685859 0.2903209 ]
 [0.2903209  0.38709454]]
Ecuador 1 vs Belgium 2:
statistic = [[ 0.        -0.0340777]
 [ 0.0340777  0.       ]],
p-value = [[1.         0.88270209]
 [0.88270209 1.        ]] 
_nobs = 2 
_ntreatments = 262 
_stand_err = [[0.13219604 0.16315668]
 [0.16315668 0.18911482]]
Ec

  4%|▍         | 2/49 [00:01<00:27,  1.72it/s]

...

Croatia 1 vs Netherlands 1:
statistic = [[ 0.          0.69160906]
 [-0.69160906  0.        ]],
p-value = [[1.         0.00123264]
 [0.00123264 1.        ]] 
_nobs = 2 
_ntreatments = 261 
_stand_err = [[0.16982873 0.14967932]
 [0.14967932 0.12635662]]
Croatia 1 vs Denmark 1:
statistic = [[ 0.          0.64419392]
 [-0.64419392  0.        ]],
p-value = [[1.00000000e+00 9.25804904e-04]
 [9.25804904e-04 1.00000000e+00]] 
_nobs = 2 
_ntreatments = 382 
_stand_err = [[0.16783872 0.13644572]
 [0.13644572 0.09521047]]
Croatia 1 vs Australia 1:
statistic = [[ 0.          1.60356794]
 [-1.60356794  0.        ]],
p-value = [[1.00000000e+00 2.63299033e-04]
 [2.63299033e-04 1.00000000e+00]] 
_nobs = 2 
_ntreatments = 115 
_stand_err = [[0.1861455  0.30093667]
 [0.30093667 0.38272133]]
Croatia 1 vs Belgium 2:
statistic = [[ 0.         0.3927857]
 [-0.3927857  0.       ]],
p-value = [[1.         0.12652225]
 [0.12652225 1.        ]] 
_nobs = 2 
_ntreatments = 179 
_stand_err = [[0.17734366 0.180916

 98%|█████████▊| 48/49 [00:30<00:00,  1.62it/s]

Greece 1 vs Germany 2:
statistic = [[ 0.          0.19806133]
 [-0.19806133  0.        ]],
p-value = [[1.         0.23763251]
 [0.23763251 1.        ]] 
_nobs = 2 
_ntreatments = 789 
_stand_err = [[0.15455022 0.11850369]
 [0.11850369 0.0648111 ]]
Greece 1 vs Austria 1:
statistic = [[ 0.         -0.22212445]
 [ 0.22212445  0.        ]],
p-value = [[1.         0.22585329]
 [0.22585329 1.        ]] 
_nobs = 2 
_ntreatments = 483 
_stand_err = [[0.15923132 0.12952113]
 [0.12952113 0.09053637]]
Greece 1 vs Croatia 1:
statistic = [[ 0.         -0.56042464]
 [ 0.56042464  0.        ]],
p-value = [[1.         0.02334492]
 [0.02334492 1.        ]] 
_nobs = 2 
_ntreatments = 211 
_stand_err = [[0.1628628  0.17346292]
 [0.17346292 0.18345156]]
Greece 1 vs Sweden 1:
statistic = [[ 0.          0.08295767]
 [-0.08295767  0.        ]],
p-value = [[1.         0.64223651]
 [0.64223651 1.        ]] 
_nobs = 2 
_ntreatments = 500 
_stand_err = [[0.15598511 0.12618899]
 [0.12618899 0.08669469]]
Sweden 1 

100%|██████████| 49/49 [00:30<00:00,  1.60it/s]

Sweden 1 vs Croatia 1:
statistic = [[ 0.         -0.64338231]
 [ 0.64338231  0.        ]],
p-value = [[1.00000000e+00 8.16654169e-04]
 [8.16654169e-04 1.00000000e+00]] 
_nobs = 2 
_ntreatments = 475 
_stand_err = [[0.08450614 0.13504503]
 [0.13504503 0.17126888]]
Sweden 1 vs Greece 1:
statistic = [[ 0.         -0.08295767]
 [ 0.08295767  0.        ]],
p-value = [[1.         0.64223651]
 [0.64223651 1.        ]] 
_nobs = 2 
_ntreatments = 500 
_stand_err = [[0.08669469 0.12618899]
 [0.12618899 0.15598511]]
